<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Gridsearching Hyperparameters


---

<img src="gridsearch_meme.png" style="width: 500px;">

### Learning Objectives
- Understand what the terms gridsearch and hyperparameter refer to.
- Understand how to manually build a gridsearching procedure.
- Apply sklearn's `GridSearchCV` object with basketball data to optimize a KNN model.
- Practice using and evaluating attributes of the gridsearch object.
- Understand the pitfalls of searching large hyperparameter spaces.
- Practice the gridsearch procedure independently optimizing regularized logistic regression.

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Big Picture: What part of the modelling process are we focusing on this morning?

---

As we looked at yesterday, one general approach you can use for modelling questions would look like this:

- STEP ONE: Cleaning, descriptive stats, correlation heatmap, plots & visualisations. Find baseline accuracy if it's a classifier.

- STEP TWO: Set up predictor matrix (X) and target array (y).  Dummify if necessary.

- STEP THREE: Train/test split and StandardScaler( )

- STEP FOUR: Use cross-validation to optimise the hyperparameters for your model. You might try different types of models at this stage as well, and you might use GridSearchCV (or any of the other CVs like RidgeCV).

- STEP FIVE: Once you're happy with your hyperparameters, fit your model on your whole training data and test it on your whole testing data.

- STEP SIX: Then you might want to evaluate the performance of the model (R2 score, accuracy, confusion matrix, etc); find the actual predictions that your model is providing and store them in a dataframe; plot your predictions against your actual target variable to visualise how well your model performed; investigate feature importance with .coef_ if you have a parametric model.

This morning, we're learning more about STEP FOUR.

<a id='intro'></a>

## What is "Gridsearching"? What are "hyperparameters"?

---

Models often have specifications that can be set. For example, when we choose a linear regression, we may decide to add a penalty to the loss function such as the Ridge or the Lasso. Those penalties require the regularization strength, alpha, to be set. 

**Model parameters are called hyperparameters.**

Hyperparameters are different than the parameters of the model resulting from a fit, such as the coefficients. The hyperparameters are set prior to the fit and determine the behavior of the model.

There are often more than one kind of hyperparamter to set for a model. For example, in the KNN algorithm, 
- we have a hyperparameter to set the number of neighbors. 
- We also have a hyperparameter for the weights: uniform or distance?

We want to know the *optimal* hyperparameter settings, the set that results in the best model evaluation. 

**The search for the optimal set of hyperparameters is called gridsearching.**

Gridsearching gets its name from the fact that we are searching over a **"grid" of parameters**. For example, imagine the `n_neighbors` hyperparameters on the x-axis and `weights` on the y-axis, and we need to test all points on the grid.

**Gridsearching uses cross-validation internally to evaluate the performance of each set of hyperparameters.** More on this later.

<a id='basketball-data'></a>

## Basketball data

---

To explore the process of gridsearching over sets of hyperparameters, we will use some basketball data. The data below has statistics for 4 different seasons of NBA basketball: 2013-2016.
- This data includes aggregate statistical data for each game. 
- The data of each game is aggregated by match for all players.
- Scraped from http://www.basketball-refrence.com

Many of the columns in the dataset represent the mean of a statistic across the last 10 games, for example. Non-target statistics are for *prior* games, they do not include information about player performance in the current game.

**We are interested in predicting whether the home team will win the game or not.** This is a classification problem.


### Load the data and create the target and predictor matrix
- The target will be a binary column of whether the home team wins.
- The predictors should be numeric statistics columns.

Exclude these columns from the predictor matrix:

    ['GameId','GameDate','GameTime','HostName',
     'GuestName','total_score','total_line','game_line',
     'winner','loser','host_wins','Season']


### STEP ONE: Cleaning, descriptive stats, correlation heatmap, plots & visualisations. Find baseline accuracy if it's a classifier.

In [54]:
data = pd.read_csv('./datasets/basketball_data.csv')

In [55]:
data.head()

,Season,GameId,GameDate,GameTime,HostName,GuestName,total_score,total_line,game_line,Host_HostRank,Host_GameRank,Guest_GuestRank,Guest_GameRank,host_win_count,host_lose_count,guest_win_count,guest_lose_count,game_behind,winner,loser,host_place_streak,guest_place_streak,hq1_avg10,hq2_avg10,hq3_avg10,hq4_avg10,hPace_avg10,heFG%_avg10,hTOV%_avg10,hORB%_avg10,hFT/FGA_avg10,hORtg_avg10,hFG_avg10,hFGA_avg10,hFG%_avg10,h3P_avg10,h3PA_avg10,h3P%_avg10,hFT_avg10,hFTA_avg10,hFT%_avg10,hORB_avg10,hDRB_avg10,hTRB_avg10,hAST_avg10,hSTL_avg10,hBLK_avg10,hTOV_avg10,hPF_avg10,hPTS_avg10,hTS%_avg10,h3PAR_avg10,hFTr_avg10,hDRB%_avg10,hTRB%_avg10,hAST%_avg10,hSTL%_avg10,hBLK%_avg10,hDRtg_avg10,gq1_avg10,gq2_avg10,gq3_avg10,gq4_avg10,gPace_avg10,geFG%_avg10,gTOV%_avg10,gORB%_avg10,gFT/FGA_avg10,gORtg_avg10,gFG_avg10,gFGA_avg10,gFG%_avg10,g3P_avg10,g3PA_avg10,g3P%_avg10,gFT_avg10,gFTA_avg10,gFT%_avg10,gORB_avg10,gDRB_avg10,gTRB_avg10,gAST_avg10,gSTL_avg10,gBLK_avg10,gTOV_avg10,gPF_avg10,gPTS_avg10,gTS%_avg10,g3PAR_avg10,gFTr_avg10,gDRB%_avg10,gTRB%_avg10,gAST%_avg10,gSTL%_avg10,gBLK%_avg10,gDRtg_avg10
0,2013,201212090LAL,2012-12-09,6:30 pm,Los Angeles Lakers,Utah Jazz,227.0,207.5,7.5,13,21,13,22,9,11,11,10,-1.5,Utah Jazz,Los Angeles Lakers,1,1,29.6,25.5,24.2,23.1,93.57,0.5059,13.28,32.11,0.2379,109.54,37.3,81.8,0.4549,8.4,23.5,0.3560,19.4,29.8,0.6611,13.9,34.3,48.2,23.1,7.9,6.0,14.5,17.2,102.4,0.5399,0.2880,0.3681,74.79,53.76,61.45,8.45,9.09,98.77,23.5,22.0,25.1,24.8,90.35,0.4791,13.96,31.63,0.2237,106.00,36.7,84.3,0.4352,7.4,18.9,0.3891,18.2,24.2,0.7611,14.6,29.0,43.6,22.4,8.0,7.1,15.4,21.0,99.0,0.5206,0.2230,0.2981,69.22,50.05,61.57,8.63,10.31,110.87
1,2013,201212100PHI,2012-12-10,7:00 pm,Philadelphia 76ers,Detroit Pistons,201.0,186.5,5.5,13,21,13,23,11,9,7,15,5.0,Philadelphia 76ers,Detroit Pistons,1,2,23.5,23.6,24.5,22.6,90.77,0.4756,10.42,23.05,0.1818,103.67,36.7,85.0,0.4328,7.2,18.6,0.3796,15.3,21.0,0.7442,10.5,29.9,40.4,22.8,8.3,5.0,11.0,20.4,95.9,0.5111,0.2205,0.2496,74.07,46.68,62.22,8.89,7.93,106.11,22.7,23.9,20.5,23.2,90.89,0.4635,14.95,27.20,0.2308,99.55,33.2,78.6,0.4249,6.0,16.9,0.3673,17.9,24.0,0.7465,11.6,31.4,43.0,20.0,5.9,5.8,15.7,19.4,90.3,0.5077,0.2144,0.3095,71.46,49.48,59.83,6.48,9.46,107.91
2,2013,201212100HOU,2012-12-10,7:00 pm,Houston Rockets,San Antonio Spurs,240.0,212.0,-7.0,12,20,13,22,9,10,17,4,-7.0,San Antonio Spurs,Houston Rockets,2,2,24.0,30.8,27.4,25.2,95.89,0.5196,13.66,25.80,0.2645,111.92,37.7,82.8,0.4566,10.3,28.3,0.3763,21.7,27.9,0.7811,11.0,31.9,42.9,22.8,8.4,4.6,15.0,19.7,107.4,0.5665,0.3437,0.3414,72.69,50.45,60.73,8.76,7.26,105.69,26.7,23.2,27.3,28.5,93.53,0.5585,13.95,21.94,0.1998,113.31,41.5,82.9,0.5044,9.0,22.8,0.3828,16.0,20.2,0.7823,8.1,34.3,42.4,25.7,7.8,4.9,14.8,17.7,108.0,0.5915,0.2743,0.2518,74.26,50.99,61.82,8.30,6.85,101.41
3,2013,201212110BRK,2012-12-11,7:00 pm,Brooklyn Nets,New York Knicks,197.0,195.5,-3.5,12,20,13,21,11,8,15,5,-3.5,New York Knicks,Brooklyn Nets,4,1,27.9,22.9,22.4,23.1,87.74,0.4884,12.86,32.49,0.2083,110.14,36.0,82.6,0.4376,8.4,24.1,0.3473,17.1,23.0,0.7380,13.9,29.2,43.1,22.5,6.6,5.8,13.6,18.7,97.5,0.5271,0.2908,0.2805,71.36,51.28,62.69,7.45,9.00,107.43,26.5,25.3,24.6,23.4,90.22,0.5121,11.32,23.64,0.1931,110.24,36.4,82.9,0.4412,11.8,30.1,0.3879,15.7,20.8,0.7519,10.4,29.7,40.1,19.0,8.5,4.7,11.7,20.1,100.3,0.5473,0.3595,0.2544,74.23,47.88,52.07,9.31,7.64,109.24
4,2013,201212110DET,2012-12-11,7:30 pm,Detroit Pistons,Denver Nuggets,195.0,203.5,-4.5,11,24,16,22,7,16,10,11,-4.0,Denver Nuggets,Detroit Pistons,1,4,24.6,26.6,23.3,25.6,90.18,0.5210,12.99,26.43,0.2430,111.00,37.6,78.3,0.4819,6.1,14.9,0.4079,18.8,26.3,0.7152,10.6,29.5,40.1,21.5,7.1,4.7,13.5,21.0,100.1,0.5585,0.1905,0.3389,70.89,49.33,57.26,7.84,7.40,105.30,24.4,29.1,23.2,24.4,92.56,0.5267,15.32,30.31,0.2257,109.28,38.5,79.4,0.4870,6.3,17.3,0.3612,17.8,25.0,0.7114,11.8,29.3,41.1,21.7,7.1,5.0,16.4,20.4,101.1,0.5605,0.2173,0.3177,68.45,50.40,56.33,7.67,7.83,114.86


In [56]:
data.columns

Index(['Season', 'GameId', 'GameDate', 'GameTime', 'HostName', 'GuestName',
       'total_score', 'total_line', 'game_line', 'Host_HostRank',
       'Host_GameRank', 'Guest_GuestRank', 'Guest_GameRank', 'host_win_count',
       'host_lose_count', 'guest_win_count', 'guest_lose_count', 'game_behind',
       'winner', 'loser', 'host_place_streak', 'guest_place_streak',
       'hq1_avg10', 'hq2_avg10', 'hq3_avg10', 'hq4_avg10', 'hPace_avg10',
       'heFG%_avg10', 'hTOV%_avg10', 'hORB%_avg10', 'hFT/FGA_avg10',
       'hORtg_avg10', 'hFG_avg10', 'hFGA_avg10', 'hFG%_avg10', 'h3P_avg10',
       'h3PA_avg10', 'h3P%_avg10', 'hFT_avg10', 'hFTA_avg10', 'hFT%_avg10',
       'hORB_avg10', 'hDRB_avg10', 'hTRB_avg10', 'hAST_avg10', 'hSTL_avg10',
       'hBLK_avg10', 'hTOV_avg10', 'hPF_avg10', 'hPTS_avg10', 'hTS%_avg10',
       'h3PAR_avg10', 'hFTr_avg10', 'hDRB%_avg10', 'hTRB%_avg10',
       'hAST%_avg10', 'hSTL%_avg10', 'hBLK%_avg10', 'hDRtg_avg10', 'gq1_avg10',
       'gq2_avg10', 'gq3_avg10', 'gq

In [57]:
data.shape

(3768, 96)

In [58]:
data.Season.value_counts()

2014    998
2016    985
2015    984
2013    801
Name: Season, dtype: int64

In [59]:
data.winner.head()

0             Utah Jazz
1    Philadelphia 76ers
2     San Antonio Spurs
3       New York Knicks
4        Denver Nuggets
Name: winner, dtype: object

In [60]:
#let's create a column called 'host_wins' which will indicate
#whether the host team won the game or not
data['host_wins'] = (data['HostName'] == data['winner']).astype(int)

In [61]:
#let's look at the baseline accuracy for this data
baseline = data['host_wins'].value_counts(normalize=True) 
baseline

1    0.594214
0    0.405786
Name: host_wins, dtype: float64

At this point, we could definitely do a bit more EDA!  We could...
- use .describe() to find some descriptive statistics
- create some plots & visualisations to better understand the shapes and relationships in our data
- use a correlation heatmap to help us find variables that could predict 'host_wins'

...but we're going to skip that for this lesson!  Be aware that if you're undertaking your own data investigation, skipping EDA is a bad idea. (Obviously.)  

### STEP TWO: Set up predictor matrix (X) and target array (y).  Dummify if necessary.

In [62]:
predictors = [c for c in data.columns if c not in ['GameId','GameDate','GameTime','HostName',
                                                   'GuestName','total_score','total_line','game_line',
                                                   'winner','loser','host_wins','Season']]
X = data[predictors]
y = data['host_wins']

In [63]:
#let's check to make sure we don't have any categorical predictors:
#if we don't have any categorical predictors, then we DON'T have to dummify
#(remember, it's normally fine to have a categorical target variable)
X.head()

,Host_HostRank,Host_GameRank,Guest_GuestRank,Guest_GameRank,host_win_count,host_lose_count,guest_win_count,guest_lose_count,game_behind,host_place_streak,guest_place_streak,hq1_avg10,hq2_avg10,hq3_avg10,hq4_avg10,hPace_avg10,heFG%_avg10,hTOV%_avg10,hORB%_avg10,hFT/FGA_avg10,hORtg_avg10,hFG_avg10,hFGA_avg10,hFG%_avg10,h3P_avg10,h3PA_avg10,h3P%_avg10,hFT_avg10,hFTA_avg10,hFT%_avg10,hORB_avg10,hDRB_avg10,hTRB_avg10,hAST_avg10,hSTL_avg10,hBLK_avg10,hTOV_avg10,hPF_avg10,hPTS_avg10,hTS%_avg10,h3PAR_avg10,hFTr_avg10,hDRB%_avg10,hTRB%_avg10,hAST%_avg10,hSTL%_avg10,hBLK%_avg10,hDRtg_avg10,gq1_avg10,gq2_avg10,gq3_avg10,gq4_avg10,gPace_avg10,geFG%_avg10,gTOV%_avg10,gORB%_avg10,gFT/FGA_avg10,gORtg_avg10,gFG_avg10,gFGA_avg10,gFG%_avg10,g3P_avg10,g3PA_avg10,g3P%_avg10,gFT_avg10,gFTA_avg10,gFT%_avg10,gORB_avg10,gDRB_avg10,gTRB_avg10,gAST_avg10,gSTL_avg10,gBLK_avg10,gTOV_avg10,gPF_avg10,gPTS_avg10,gTS%_avg10,g3PAR_avg10,gFTr_avg10,gDRB%_avg10,gTRB%_avg10,gAST%_avg10,gSTL%_avg10,gBLK%_avg10,gDRtg_avg10
0,13,21,13,22,9,11,11,10,-1.5,1,1,29.6,25.5,24.2,23.1,93.57,0.5059,13.28,32.11,0.2379,109.54,37.3,81.8,0.4549,8.4,23.5,0.3560,19.4,29.8,0.6611,13.9,34.3,48.2,23.1,7.9,6.0,14.5,17.2,102.4,0.5399,0.2880,0.3681,74.79,53.76,61.45,8.45,9.09,98.77,23.5,22.0,25.1,24.8,90.35,0.4791,13.96,31.63,0.2237,106.00,36.7,84.3,0.4352,7.4,18.9,0.3891,18.2,24.2,0.7611,14.6,29.0,43.6,22.4,8.0,7.1,15.4,21.0,99.0,0.5206,0.2230,0.2981,69.22,50.05,61.57,8.63,10.31,110.87
1,13,21,13,23,11,9,7,15,5.0,1,2,23.5,23.6,24.5,22.6,90.77,0.4756,10.42,23.05,0.1818,103.67,36.7,85.0,0.4328,7.2,18.6,0.3796,15.3,21.0,0.7442,10.5,29.9,40.4,22.8,8.3,5.0,11.0,20.4,95.9,0.5111,0.2205,0.2496,74.07,46.68,62.22,8.89,7.93,106.11,22.7,23.9,20.5,23.2,90.89,0.4635,14.95,27.20,0.2308,99.55,33.2,78.6,0.4249,6.0,16.9,0.3673,17.9,24.0,0.7465,11.6,31.4,43.0,20.0,5.9,5.8,15.7,19.4,90.3,0.5077,0.2144,0.3095,71.46,49.48,59.83,6.48,9.46,107.91
2,12,20,13,22,9,10,17,4,-7.0,2,2,24.0,30.8,27.4,25.2,95.89,0.5196,13.66,25.80,0.2645,111.92,37.7,82.8,0.4566,10.3,28.3,0.3763,21.7,27.9,0.7811,11.0,31.9,42.9,22.8,8.4,4.6,15.0,19.7,107.4,0.5665,0.3437,0.3414,72.69,50.45,60.73,8.76,7.26,105.69,26.7,23.2,27.3,28.5,93.53,0.5585,13.95,21.94,0.1998,113.31,41.5,82.9,0.5044,9.0,22.8,0.3828,16.0,20.2,0.7823,8.1,34.3,42.4,25.7,7.8,4.9,14.8,17.7,108.0,0.5915,0.2743,0.2518,74.26,50.99,61.82,8.30,6.85,101.41
3,12,20,13,21,11,8,15,5,-3.5,4,1,27.9,22.9,22.4,23.1,87.74,0.4884,12.86,32.49,0.2083,110.14,36.0,82.6,0.4376,8.4,24.1,0.3473,17.1,23.0,0.7380,13.9,29.2,43.1,22.5,6.6,5.8,13.6,18.7,97.5,0.5271,0.2908,0.2805,71.36,51.28,62.69,7.45,9.00,107.43,26.5,25.3,24.6,23.4,90.22,0.5121,11.32,23.64,0.1931,110.24,36.4,82.9,0.4412,11.8,30.1,0.3879,15.7,20.8,0.7519,10.4,29.7,40.1,19.0,8.5,4.7,11.7,20.1,100.3,0.5473,0.3595,0.2544,74.23,47.88,52.07,9.31,7.64,109.24
4,11,24,16,22,7,16,10,11,-4.0,1,4,24.6,26.6,23.3,25.6,90.18,0.5210,12.99,26.43,0.2430,111.00,37.6,78.3,0.4819,6.1,14.9,0.4079,18.8,26.3,0.7152,10.6,29.5,40.1,21.5,7.1,4.7,13.5,21.0,100.1,0.5585,0.1905,0.3389,70.89,49.33,57.26,7.84,7.40,105.30,24.4,29.1,23.2,24.4,92.56,0.5267,15.32,30.31,0.2257,109.28,38.5,79.4,0.4870,6.3,17.3,0.3612,17.8,25.0,0.7114,11.8,29.3,41.1,21.7,7.1,5.0,16.4,20.4,101.1,0.5605,0.2173,0.3177,68.45,50.40,56.33,7.67,7.83,114.86


### STEP THREE: Train/test split and StandardScaler( ).  In this case, instead of using train_test_split, we're going to use the most recent season as our testing data (2016 data), and previous seasons as our training data.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
data['Season'].value_counts()

In [ ]:
#create your training and testing sets
X_train = X[data['Season'].isin([2013,2014,2015])]  #or, you could use X[data['Season]<2016]
X_test = X[data['Season'] == 2016]

y_train = y[data['Season'].isin([2013,2014,2015])]
y_test = y[data['Season'] == 2016]

In [ ]:
#standardise your predictor matrices
ss = StandardScaler()
ss.fit(X_train)
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

### STEP FOUR: Use cross-validation to optimise the hyperparameters for your model. You might try different types of models at this stage as well, and you might use GridSearchCV (or any of the other CVs like RidgeCV).

Below we can fit a default `KNeighborsClassifier` to predict 'host_wins'.

We can use cross-validation with our training data to see how well it performs.

In [91]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [92]:
#set up a default KNN model and cross-validate it on the training data
#use 5 cross-validation folds
knn = KNeighborsClassifier()
cross_val_score(knn, X_train_ss, y_train, cv=5)

array([0.59066427, 0.57989228, 0.58527828, 0.57553957, 0.61690647])

In [93]:
#find the mean for your cross_val_scores
knn_cv_accuracy = cross_val_score(knn, X_train_ss, y_train, cv=5).mean()
print('Mean cross-validated accuracy for default knn:',knn_cv_accuracy)
print('Baseline accuracy:',baseline)

Mean cross-validated accuracy for default knn: 0.589656174521783
Baseline accuracy: 1    0.594214
0    0.405786
Name: host_wins, dtype: float64


Our default KNN performs quite poorly on the test data. But what if we changed the number of neighbors? The weighting? The distance metric?

These are all hyperparameters of the KNN. How would we do this manually? We would need to evaluate on the training data the set of hyperparameters that perform best, and then use this set of hyperparameters to fit the final model and score on the testing set.

#### Gridsearch pseudocode for our KNN

```python
accuracies = {}
for k in neighbors_to_test:
    for w in weightings_to_test:
        for d in distance_metrics_to_test:
            hyperparam_set = (k, w, d)
            knn = KNeighborsClassifier(n_neighbors=n, weights=w, metric=d)
            cv_accuracies = cross_val_score(knn, X_train, y_train, cv=5)
            accuracies[hyperparam_set] = np.mean(cv_accuracies)
```

In the pseudocode above, we would find the key in the dictionary (a hyperparameter set) that has the larget value (mean cross-validated accuracy).



#### Using `GridSearchCV`

This would be an annoying process to have to do manually. Luckily sklearn comes with a convenience class for performing gridsearch:

```python
from sklearn.model_selection import GridSearchCV
```

The `GridSearchCV` has a handful of important arguments:

| Argument | Description |
| --- | ---|
| **`estimator`** | The sklearn instance of the model to fit on |
| **`param_grid`** | A dictionary where keys are hyperparameters for the model and values are lists of values to test |
| **`cv`** | The number of internal cross-validation folds to run for each set of hyperparameters |
| **`n_jobs`** | How many cores to use on your computer to run the folds (-1 means use all cores) |
| **`verbose`** | How much output to display (0 is none, 1 is limited, 2 is printouts for every internal fit) |


Below is an example for how one might set up the gridsearch for our KNN:

```python
knn_parameters = {
    'n_neighbors':[1,3,5,7,9],
    'weights':['uniform','distance']
}

knn_gridsearcher = GridSearchCV(KNeighborsClassifier(), knn_parameters, cv=4, verbose=1)
knn_gridsearcher.fit(X_train, y_train)
```

**Try out the sklearn gridsearch below on the training data.**

In [94]:
from sklearn.model_selection import GridSearchCV

In [96]:
knn_params = {
    'n_neighbors': [5,9,15,25,40,50,60],
    'weights':['uniform','distance'],
    'metric':['euclidean','manhattan']}

knn_gridsearch = GridSearchCV(KNeighborsClassifier(), 
                              knn_params, 
                              n_jobs=-1, cv=5, verbose=1)

knn_gridsearch.fit(X_train_ss, y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV]  metric=euclidean, n_neighbors=5, weights=uniform, score=0.5906642728904847, total=   0.5s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV]  metric=euclidean, n_neighbors=5, weights=uniform, score=0.585278276481149, total=   0.6s
[CV] metric=euclidean, n_neighbors=5, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=5, weights=uniform, score=0.5755395683453237, total=   0.5s
[CV]  metric=euclidean, n_neighbors=5, weights=uniform, score=0.5798922800718133, total=   0.6s
[CV] metric=euclidean, n_neighbors=5, weights=distance ...............
[CV] metric=euclidean, n_neighbors=5, weights=distance ...

[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   15.5s


[CV]  metric=euclidean, n_neighbors=15, weights=uniform, score=0.6223021582733813, total=   0.6s
[CV] metric=euclidean, n_neighbors=15, weights=distance ..............
[CV]  metric=euclidean, n_neighbors=15, weights=distance, score=0.5960502692998204, total=   0.6s
[CV] metric=euclidean, n_neighbors=15, weights=distance ..............
[CV]  metric=euclidean, n_neighbors=15, weights=distance, score=0.6157989228007181, total=   0.5s
[CV] metric=euclidean, n_neighbors=25, weights=uniform ...............
[CV]  metric=euclidean, n_neighbors=15, weights=distance, score=0.6032315978456014, total=   0.6s
[CV] metric=euclidean, n_neighbors=25, weights=uniform ...............
[CV]  metric=euclidean, n_neighbors=15, weights=distance, score=0.5971223021582733, total=   0.6s
[CV] metric=euclidean, n_neighbors=25, weights=uniform ...............
[CV]  metric=euclidean, n_neighbors=15, weights=distance, score=0.6223021582733813, total=   0.6s
[CV] metric=euclidean, n_neighbors=25, weights=uniform ...

[CV]  metric=manhattan, n_neighbors=5, weights=uniform, score=0.579136690647482, total=   0.6s
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=5, weights=uniform, score=0.6187050359712231, total=   0.6s
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=0.6104129263913824, total=   0.5s
[CV] metric=manhattan, n_neighbors=5, weights=distance ...............
[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=0.6157989228007181, total=   0.7s
[CV] metric=manhattan, n_neighbors=9, weights=uniform ................
[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=0.579136690647482, total=   0.7s
[CV] metric=manhattan, n_neighbors=9, weights=uniform ................
[CV]  metric=manhattan, n_neighbors=5, weights=distance, score=0.59245960502693, total=   0.6s
[CV] metric=manhattan, n_neighbors=9, weights=uniform ...............

[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  1.4min


[CV]  metric=manhattan, n_neighbors=50, weights=uniform, score=0.6373429084380611, total=   0.6s
[CV] metric=manhattan, n_neighbors=50, weights=uniform ...............
[CV]  metric=manhattan, n_neighbors=50, weights=uniform, score=0.6104129263913824, total=   0.6s
[CV] metric=manhattan, n_neighbors=50, weights=distance ..............
[CV]  metric=manhattan, n_neighbors=50, weights=uniform, score=0.6319569120287253, total=   0.7s
[CV] metric=manhattan, n_neighbors=50, weights=distance ..............
[CV]  metric=manhattan, n_neighbors=50, weights=uniform, score=0.6115107913669064, total=   0.7s
[CV] metric=manhattan, n_neighbors=50, weights=distance ..............
[CV]  metric=manhattan, n_neighbors=50, weights=uniform, score=0.6420863309352518, total=   0.7s
[CV] metric=manhattan, n_neighbors=50, weights=distance ..............
[CV]  metric=manhattan, n_neighbors=50, weights=distance, score=0.6355475763016158, total=   0.7s
[CV] metric=manhattan, n_neighbors=50, weights=distance ......

[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_neighbors': [5, 9, 15, 25, 40, 50, 60], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

Great!  We've fit our GridSearch.

#### Examing the results of GridSearch( )

Once the gridsearch has fit (this can take awhile!) we can pull out a variety of information and useful objects from the gridsearch object, stored as attributes:

| Property | Use |
| --- | ---|
| **`results.param_grid`** | Displays parameters searched over. |
| **`results.best_score_`** | Best mean cross-validated score achieved. |
| **`results.best_estimator_`** | Reference to model with best score.  Is usable / callable. |
| **`results.best_params_`** | The parameters that have been found to perform with the best score. |
| **`results.grid_scores_`** | Display score attributes with corresponding parameters. | 

**Print out the best score found in the search.**

In [97]:
#print out the best mean cross-validated accuracy from the gridsearch
#hopefully this should be much better than our default mean cross-validated accuracy 
knn_gridsearch.best_score_

0.6266618756737333

In [74]:
print('Mean cross-validated accuracy for default knn:',knn_cv_accuracy)
print('Baseline accuracy:',baseline)

Mean cross-validated accuracy for default knn: 0.589656174521783
Baseline accuracy: 1    0.594214
0    0.405786
Name: host_wins, dtype: float64


**Print out the set of hyperparameters that achieved the best score.**

In [75]:
#print out your best hyperparameters
knn_gridsearch.best_params_

{'metric': 'manhattan', 'n_neighbors': 50, 'weights': 'uniform'}

In [102]:
md = knn_gridsearch.best_estimator_

In [103]:
dir(md)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_init_params',
 '_pairwise',
 '_tree',
 '_y',
 'algorithm',
 'classes_',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_jobs',
 'n_neighbors',
 'outputs_2d_',
 'p',
 'predict',
 'predict_proba',
 'radius',
 'score',
 'set_params',
 'weights']

### STEP FIVE: Once you're happy with your hyperparameters, fit your model on your whole training data and test it on your whole testing data.  (When you use a gridsearch's `.best_estimator_`, it will already have fit a model with the best hyperparameters on your training data, so all you have to do is score it on your testing data.)

**Assign the best fit model (`best_estimator_`) to a variable and score it on the test data.**

Compare this model to the baseline accuracy and your default KNN.

In [76]:
#assign your best_estimator_ to the variable, then use .score( ) on your testing data
best_knn = knn_gridsearch.best_estimator_
best_knn.score(X_test_ss, y_test)

0.6467005076142132

### STEP SIX: Then you might want to evaluate the performance of the model (R2 score, accuracy, confusion matrix, etc); find the actual predictions that your model is providing and store them in a dataframe; plot your predictions against your actual target variable to visualise how well your model performed; investigate feature importance with .coef_ if you have a parametric model.

In [105]:
sum(y_test)

594

In [107]:
#there's lots of stuff you can do to follow up and investigate when you've found your best model,
#but let's just look at some different ways to assess a classifier now using what you learned yesterday:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

predictions = best_knn.predict(X_test_ss)
confusion = confusion_matrix(y_test,predictions,labels=[1,0])
pd.DataFrame(confusion, 
             columns=['predicted_home_win','predicted_home_loss'], 
             index=['True_home_win','True_home_loss'])

,predicted_home_win,predicted_home_loss
True_home_win,501,93
True_home_loss,255,136


In [78]:
#recall is concerned with how much you catch all the positives:
#it's more important with cancer detection tests, for example: 
#     "Let's make sure we RECALL all the patients with positives to do further testing!" 
#     "Yes!  But let's make sure we're SENSITIVE about how we tell them they might have cancer."
#it's the true positives, divided by all the actual positives
501/(501+93)

0.8434343434343434

In [79]:
#let's check that:
recall_score(y_test,predictions)

0.8434343434343434

In [108]:
#precision is concerned with how precisely you predict positives; in other words,
#when you predict a positive, are you pretty sure about that prediction?
#    "Whhaaattttt. An email from the Home Office went to my spam filter?!
#    "Spam filters are great, but they damn sure better be PRECISE about what emails they think are spam!"
#it's the true positives, divided by all the predicted positives
501/(501+255)

0.6626984126984127

In [117]:
#let's check that:
precision_score(y_test,predictions)

0.6626984126984127

In [110]:
#the F1 score is the harmonic mean between these two metrics:
f1_score(y_test,predictions)

0.7422222222222222

**Checkout (in pair):** How can you modify the recall or the precision?

In [114]:
predictions_proba = best_knn.predict_proba(X_test_ss)

In [115]:
new_predictions = [int(p[1]>.3) for p in predictions_proba]

In [116]:
recall_score(y_test,new_predictions)

0.9983164983164983

In [118]:
precision_score(y_test,new_predictions)

0.6063394683026585

<a id='practice'></a>

## Independent practice: gridsearch regularization penalties with logistic regression

---

Logistic regression models can also apply the Lasso and Ridge penalties. The `LogisticRegression` class takes these regularization-relevant hyperparameters:

| Argument | Description |
| --- | ---|
| **`penalty`** | `'l1'` for Lasso, `'l2'` for Ridge |
| **`solver`** | Must be set to `'liblinear'` for the Lasso penalty to work. |
| **`C`** | The regularization strength. Equivalent to `1./alpha` |

**You should:**
1. Fit and validate the accuracy of a default logistic regression on the basketball data.
- Perform a gridsearch over different regularization strengths and Lasso and Ridge penalties.
- Compare the accuracy on the test set of your optimized logistic regression to the baseline accuracy and the default model.
- Look at the best parameters found. What was chosen? What does this suggest about our data?
- Look at the coefficients and associated predictors for your optimized model. What appears to be the most important predictors of winning the game?


In [86]:
from sklearn.linear_model import LogisticRegression

In [87]:
#set up a logistic regression model, 
#and find its cross-validated scores for your training data
#you should get accuracies higher than 0.6, which is better than KNN!
lr = LogisticRegression()
cross_val_score(lr, X_train_ss, y_train, cv=5)

array([0.64452424, 0.62118492, 0.68402154, 0.64028777, 0.65827338])

In [88]:
#find the mean of those cross-validated scores:
cross_val_score(lr, X_train_ss, y_train, cv=5).mean()

0.649658370251734

In [119]:
# Set up the parameters. 
# Use a list with 'l1' and 'l2' for the penalties,
# Use a list with 'liblinear' for the solver,
# Use a logspace from -3 to 0, with 50 different values

# fill the dictionary of parameters
gs_params = {'penalty':['l1','l2'],
             'solver':['liblinear'],
             'C':np.logspace(-3,0,50)}

#create your gridsearch object
lr_gridsearch = GridSearchCV(LogisticRegression(), 
                             gs_params, 
                             n_jobs=-1, cv=5, verbose=1)


In [ ]:
#fit your gridsearch object on your training data
lr_gridsearch.fit(X_train_ss, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
# find the best mean cross-validated score that your gridsearch found:
# (this should be better than the mean cross-validated score for your default logistic regression above)
lr_gridsearch.best_score_

In [ ]:
# find the best hyperparameters that your gridsearch found:
lr_gridsearch.best_params_

In [ ]:
# assign the best estimator to a variable:
best_lr = lr_gridsearch.best_estimator_

In [ ]:
# score your best estimator on the testing data:
best_lr.score(X_test_ss, y_test)

### Let's analyse the features importances

In [ ]:
# create a dataframe to look at the coefficients
coef_df = pd.DataFrame({'coef': best_lr.coef_[0],
                        'feature': X.columns,
                        'abs_coef': np.abs(best_lr.coef_[0])})

coef_df.head()

In [ ]:
# sort by absolute value of coefficient (magnitude)
coef_df.sort_values('abs_coef', ascending=False, inplace=True)
coef_df.head()

## KEY TAKEAWAYS:

- You always want to use your training data to search for your best hyperparameters! You can do this with GridSearchCV, or with other sklearn objects like RidgeCV, LassoCV, ElasticNetCV, or LogisticRegressionCV.  


- You instantiate GridSearchCV with:
    - a model
    - a dictionary for that model's parameters
    - the number of cross-validation folds you want it to perform (`cv=`)
    - how many cores to use on your computer for this job (`n_jobs=`)
    - whether you want your model to give you some print-outs as it works (`verbose=`)
    

- Once you've instantiated the GridSearch object, you can fit it on your training data


- Once it's finished searching, you can access some useful attributes:
    - `.best_score_`, to find the mean cross-validated score of the best estimator
    - `.best_params_`, to find the best hyperparameters 
    - `.best_estimator_`, which you will assign to a variable in order to use `.score()`, `.predict()`, `.coef_`, etc

## PROGRESS CHECKPOINT:

- EVERYONE should be able to:
    - give an example of a hyperparameter
    - explain the basic intuition behind how GridSearch works
    - copy and paste the GridSearching from this lesson and adapt it to a different lab to find the best hyperparameters for a model



- MANY of you should be able to:
    -  use your best estimator from the gridsearch to evaluate model effectiveness and investigate feature importance
    -  come up with reasonable parameter dictionaries by yourself for KNN and Logistic Regression
    -  create a confusion matrix using your predictions from your best estimator, and understand how to interpret it


- SOME of you should be able to:
    - look at the documentation for GridSearchCV on sklearn's website, and investigate the different options that are available (for example, the attribute `.cv_results_`)
    - create a ROC curve and a Precision-Recall curve for your predictions, and be able to interpret them
    